In [ ]:
import matplotlib.pyplot as plt
from subprocess import Popen, PIPE
from shutil import copyfileobj
import numpy as np
from math import sqrt

In [ ]:
def create_uniform_clusters(n):
    limit = int(sqrt(n/2))
    points = []
    for i in range(1, limit + 1):
        for j in range(1, limit + 1):
            points.append([i, j])
            points.append([limit + i + 6, j])
    return points

In [ ]:
def create_low_dispersion_clusters(n):
    points = create_uniform_clusters(n)
    X_jitter = np.random.normal(0, 0.1, len(points))
    Y_jitter = np.random.normal(0, 0.1, len(points))
    for i in range(len(points)):
        points[i][0] += X_jitter[i]
        points[i][1] += Y_jitter[i]
    return points

In [ ]:
def create_high_dispersion_clusters(n):
    points = create_uniform_clusters(n)
    X_jitter = np.random.normal(0, 1.0, len(points))
    Y_jitter = np.random.normal(0, 1.0, len(points))
    for i in range(len(points)):
        points[i][0] += X_jitter[i]
        points[i][1] += Y_jitter[i]
    return points

In [ ]:
def points_to_input(points):
    str_input = str(len(points)) + '\n'
    for point in points:
        str_input += str(point[0]) + ' ' + str(point[1]) + '\n'
    return str_input

In [ ]:
test = create_low_dispersion_clusters(1000)
test = np.array(test)
X = test[:,0]
Y = test[:,1]
plt.scatter(X, Y)
plt.show()
print(len(test))

In [ ]:
test_input = points_to_input(test)
#print(test_input)

In [ ]:
executablePath = '../../clustering'
mstMode = 'Prim'
inconsistencyModeRange = ['StandardDeviation', 'Factor']
neighborhoodDepthRange = ['3']
inconsistencyParameterRange = [str(x) for x in np.linspace(2, 5, 11)]

In [ ]:
for inconsistencyMode in inconsistencyModeRange:
    for neighborhoodDepth in neighborhoodDepthRange:
        for inconsistencyParameter in inconsistencyParameterRange:
            with Popen([executablePath, mstMode, inconsistencyMode, neighborhoodDepth, inconsistencyParameter], stdin=PIPE, stdout=PIPE) as proc:
                proc.stdin.write(test_input.encode('UTF-8'))
                proc.stdin.flush()
                output = proc.stdout.read().decode('UTF-8')
            C = [int(x) for x in output.split()]
            clusters = len(set(C))
            scatter = plt.scatter(X, Y, c=C)
            plt.title('MST={0}, Inconsistency={1}({3}), Neighborhood depth={2}, Clusters={4}'.format(mstMode, inconsistencyMode, neighborhoodDepth, inconsistencyParameter, clusters))
            plt.colorbar(scatter)
            plt.show()
#print(C[:10])